# Converting geotagged tweets to GEOJSON
(Reference: https://github.com/INFO-4602-5602/final-project-tweetsonamap/issues/4)

In [1]:
import pymongo, json, pprint, urllib.request, os.path, sys

Connect to the DB

    client = pymongo.MongoClient('mongodb://epic-analytics.cs.colorado.edu')
    db = client.matthew
    print(db.collection_names())
    tweets = db.tweets
    print(tweets.count(), "tweets")

In [2]:
client = pymongo.MongoClient('localhost')
tweets = client['matthew'].tweets

In [3]:
cursor = tweets.find({'geo' : {"$exists":True}}).limit(10000).batch_size(10)

In [84]:
idx = 0;
feats = []
for t in cursor:
    tweet_id = t['id'].split(":")[2]
    if os.path.exists("/data/www/jennings/infovis/map_images/"+tweet_id+".jpg"):
        jsonObj = { "type" : "Feature",
                 "geometry" : {"type" : "Point", "coordinates" : list(reversed(t['geo']['coordinates']))},
                 "properties" : {  "id" : tweet_id,
                                   "user" : t['actor']['preferredUsername'],
                                   "text" : t['body'],
                                   "time" : 10, #placeholder
                                   "topic" : 1 #placeholder
           }
         }
        feats.append(jsonObj)

KeyError: 'geo'

In [82]:
with open('/data/www/jennings/infovis-insta10000.geojson','w') as outFile:
    json.dump({'type':"FeatureCollection","features":feats},outFile)

## For Twitter

In [9]:
geo_image_tweets = {
    "verb":"post", # original tweet
    "twitter_extended_entities.media.0":{"$exists":True}, # at least 1 media entity
    "twitter_extended_entities.media": {"$all":[{"$elemMatch": { "type": "photo" }}]}, # all media entities are photos
    "location.geo.coordinates":{"$exists":True} # has geolocation (not necessarily a point)
}
cursor = tweets.find(geo_image_tweets).limit(10)

In [94]:
idx = 0;
feats = []
for t in cursor:
    tweet_id = t['id'].split(":")[2]
    if os.path.exists("/data/www/jennings/infovis/map_images/"+tweet_id+".jpg"):
        if t['location']['geo']['type'] == "Point":
            coords = t['location']['geo']['coordinates']
            jsonObj = { "type" : "Feature",
                     "geometry" : {"type" : "Point", "coordinates" : coords},
                     "properties" : {  "id" : tweet_id,
                                       "user" : t['actor']['preferredUsername'],
                                       "text" : t['body'],
                                       "time" : 10, #Stubbed,
                                       "topic" : 1 #Stubbed
               }
             }
            feats.append(jsonObj)

In [95]:
with open('/data/www/jennings/infovis-twitter10000.geojson','w') as outFile:
    json.dump({'type':"FeatureCollection","features":feats},outFile)

In [10]:
from pprint import pprint
for t in cursor:
    pprint(t)

{'_id': ObjectId('58c1e007d8991e9a470000a9'),
 'actor': {'displayName': 'Boogie',
           'favoritesCount': 7945,
           'followersCount': 687,
           'friendsCount': 645,
           'id': 'id:twitter.com:314933045',
           'image': 'https://pbs.twimg.com/profile_images/704460627784425473/apjRkujp_normal.jpg',
           'languages': ['en'],
           'link': 'http://www.twitter.com/ZayB3_',
           'links': [{'href': None, 'rel': 'me'}],
           'listedCount': 2,
           'location': {'displayName': "I'm everywhere....",
                        'objectType': 'place'},
           'objectType': 'person',
           'postedTime': datetime.datetime(2011, 6, 11, 1, 44, 14),
           'preferredUsername': 'ZayB3_',
           'statusesCount': 25329,
           'summary': '#RIPCHENK #RIPDRUGZ #StayHumble',
           'twitterTimeZone': 'Quito',
           'utcOffset': '-18000',
           'verified': False},
 'body': 'Mood: 3 months from now when we touchdown in Jama